# Importing packages

In [ ]:
from scipy.stats import kurtosis
from scipy.signal import lfilter
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, scale
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
#from nitime.algorithms.autoregressive import AR_est_YW
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import hiplot as hip
sys.path.append('..')
sys.path.append('../utils/')
from utils.EDA import*

In [ ]:
plt.style.use('default')

plt.rcParams.update({
    'font.size': 20,
    'axes.linewidth': 2,
    'axes.titlesize': 20,
    'axes.edgecolor': 'black',
    'axes.labelsize': 18,
    'axes.grid': True,
    'lines.linewidth': 1.5,
    'lines.markersize': 6,
    'figure.figsize': (20, 8),
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'font.family': 'Times New Roman',
    'legend.fontsize': 13,
    'legend.framealpha': 0.8,
    'legend.edgecolor': 'black',
    'legend.shadow': False,
    'legend.fancybox': True,
    'legend.frameon': True,
})

In [ ]:
path_to_dataset = "Datasets/OP6_Features.parquet"
df = pd.read_parquet(path_to_dataset)

# Visualizing the data after the FE step

In [ ]:
selected_columns = ['Y_Rolling Variance','Y_Rolling Energy Entropy','X_D3','Y_D1', 'Z_D2','Z_Rolling Energy Entropy','Y_Rolling Median']

plot_scatter_matrix_FE(df, machine='M02', process='OP06', cols=selected_columns, sample_frac=0.1, random_state=42)

In [ ]:
plot_vars = ['X_axis','Y_axis','Z_axis','Machine','Label'] 
visualize_with_hiplot(df[plot_vars].sample(frac=0.01,random_state=0))

In [ ]:
plot_vars = ['Z_D3', 'Z_D2', 'Y_D3', 'Y_D2', 'X_D3','Machine','Label','Z_Rolling Energy Entropy', 'Y_Rolling RMS'] 
visualize_with_hiplot(df[plot_vars].sample(frac=0.01,random_state=0))

In [ ]:
#Z_D3, Z_D2, Y_D3, Y_D2, X_D3

In [ ]:
features = ['X_axis', 'X_Rolling Mean', 'X_Rolling Std', 'X_Rolling Max',
       'X_Rolling Min', 'X_Rolling Median', 'X_Rolling Variance',
       'X_Rolling Skewness', 'X_Rolling Kurtosis', 'X_Rolling RMS',
       'X_Rolling Impulse Factor', 'X_Rolling Margin Factor', 'Y_axis',
       'Y_Rolling Mean', 'Y_Rolling Std', 'Y_Rolling Max', 'Y_Rolling Min',
       'Y_Rolling Median', 'Y_Rolling Variance', 'Y_Rolling Skewness',
       'Y_Rolling Kurtosis', 'Y_Rolling RMS', 'Y_Rolling Impulse Factor',
       'Y_Rolling Margin Factor', 'Z_axis', 'Z_Rolling Mean', 'Z_Rolling Std',
       'Z_Rolling Max', 'Z_Rolling Min', 'Z_Rolling Median',
       'Z_Rolling Variance', 'Z_Rolling Skewness', 'Z_Rolling Kurtosis',
       'Z_Rolling RMS', 'Z_Rolling Impulse Factor', 'Z_Rolling Margin Factor',
       'X_Rolling Energy', 'X_Rolling Energy Entropy',
       'X_Rolling Normalized Energy', 'Y_Rolling Energy',
       'Y_Rolling Energy Entropy', 'Y_Rolling Normalized Energy',
       'Z_Rolling Energy', 'Z_Rolling Energy Entropy',
       'Z_Rolling Normalized Energy', 'X_D1', 'X_D2', 'X_D3', 'X_A3', 'Y_D1',
       'Y_D2', 'Y_D3', 'Y_A3', 'Z_D1', 'Z_D2', 'Z_D3', 'Z_A3', 'X_Jerk',
       'Y_Jerk', 'Z_Jerk']
X = df[features]

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=features)

# Adding labels back for later use in plotting
#X_scaled = pd.concat([X_scaled, df1['Label']], axis=1)

In [ ]:
def plot_pca_cumulative_variance(X_scaled, n_components=None):
    """
    Fit a PCA model to the given scaled data and plot the cumulative explained variance.

    Parameters:
    - X_scaled: The scaled input data (e.g., a NumPy array or DataFrame).
    - n_components: Number of PCA components to consider (default is None, which means all).
    """
    # Initialize PCA model
    pca = PCA(n_components=n_components)
    
    # Fit PCA model to the scaled data
    pca.fit(X_scaled)
    
    # Calculate the cumulative explained variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    
    # Create a plot of the cumulative explained variance
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('PCA - Cumulative Variance Explained by Components')
    plt.grid(True)
    plt.show()

In [ ]:
plot_pca_cumulative_variance(X_scaled)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA

def pca_analysis_with_inverse_plot(X_scaled, n_components, feature_names):
    """
    Perform PCA, plot a heatmap showing the contributions of each feature to the principal components,
    and return the DataFrame of the inverted PCA transformation.

    Parameters:
    - X_scaled: The scaled input data (NumPy array or DataFrame).
    - n_components: Number of PCA components to retain.
    - feature_names: List of original feature names.

    Returns:
    - DataFrame of the inverted PCA transformation approximating original features.
    """
    pca = PCA(n_components=n_components)

    # Fit PCA and transform the scaled data into principal components
    principal_components = pca.fit_transform(X_scaled)

    # Inverse transform to approximate the original features
    inverted_data = pca.inverse_transform(principal_components)
    inverted_df = pd.DataFrame(inverted_data, columns=feature_names)

    # Identify the features contributing the most to each principal component
    loading_matrix = np.abs(pca.components_)
    top_contributors = {}
    for idx, component in enumerate(loading_matrix):
        top_features_idx = component.argsort()[::-1]  # Sort in descending order
        top_features = [feature_names[i] for i in top_features_idx]
        top_contributors[f'PC{idx + 1}'] = top_features

    # Create a DataFrame of top feature contributions
    contributions_df = pd.DataFrame.from_dict(top_contributors, orient='index', columns=[f'Top_Feature_{i + 1}' for i in range(len(feature_names))])

    return inverted_df, contributions_df

In [ ]:
n_components = 40  # Choose the desired number of principal components
inverted_df, contributions_df = pca_analysis_with_inverse_plot(X_scaled, n_components, feature_names = features)
print("Approximated Original Features (Inverted PCA):")

In [ ]:
çakçlkdçlkdçdç

In [ ]:
inverted_df.head()

In [ ]:
print("\nTop Feature Contributions for Each Principal Component:")
contributions_df.head()

In [ ]:
all_features_series = pd.Series(contributions_df.values.flatten())

In [ ]:
# Apply PCA and retain the first 30 principal components
n_components = 40
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(X_scaled)

# Create a DataFrame for the principal components
pca_columns = [f'PC{i+1}' for i in range(n_components)]
pca_df = pd.DataFrame(principal_components, columns=pca_columns)

In [ ]:
df.columns

In [ ]:
pca_df = pd.concat[[pca_df,]]

In [ ]:
# Perform the inverse transformation to approximate the original data
reconstructed_data = pca.inverse_transform(principal_components)

# Convert the reconstructed data back to a DataFrame
reconstructed_df = pd.DataFrame(reconstructed_data, columns=features)

In [ ]:
# Compute the Mean Absolute Error (MAE)
mae = mean_absolute_error(X_scaled, reconstructed_data)
# Print the MAE
print(f"Reconstruction Mean Absolute Error (MAE): {mae}")

In [ ]:
# Undoing the standardizetian
reconstructed_original_scale = scaler.inverse_transform(reconstructed_data)
reconstructed_original_df = pd.DataFrame(reconstructed_original_scale, columns=features)

In [ ]:
reconstructed_original_df

In [ ]:
df.columns

In [ ]:
reconstructed_original_df = pd.concat([df[['Time', 'Month', 'Year', 'Machine', 'Process', 'Label', 'Unique_Code',
       'Period']],reconstructed_original_df], axis=1)

In [ ]:
selected_columns = ['Y_Rolling Variance','Y_Rolling Energy Entropy','X_D3','Y_D1', 'Z_D2','Z_Rolling Energy Entropy','Y_Rolling Median']

plot_scatter_matrix_FE(df, machine='M01', process='OP06', cols=selected_columns, sample_frac=0.1, random_state=42)

In [ ]:
selected_columns = ['Y_Rolling Variance','Y_Rolling Energy Entropy','X_D3','Y_D1', 'Z_D2','Z_Rolling Energy Entropy','Y_Rolling Median']

plot_scatter_matrix_FE(reconstructed_original_df, machine='M01', process='OP06', cols=selected_columns, sample_frac=0.1, random_state=42)